# 01. 데이터 파이프라인

## 주요 피처
| 피처 | 설명 |
|------|------|
| drop_distance_from_path | 비행기 경로에서 낙하 지점까지 수직 거리 |
| early_enemy_density | 낙하 직후 반경 500m 내 적 수 |
| rotation_timing_score | 자기장 선점 vs 후행 비율 (0=선점, 1=후행) |
| vehicle_use_ratio | 차량 이동 비율 |
| bluezone_exposure_ratio | 블루존 체류 비율 |
| safezone_proximity_mean | 안전구역 중심까지 평균 거리 |
| safezone_edge_ratio | 안전구역 반경 대비 상대 거리 |
| altitude_variance | 고도 변화량 (지형 활용도) |
| max_vehicle_distance | 낙하 지점 대비 최대 차량 이동 거리 |

**파일 구조:** `temp_parquet_files_{server}_{ymd}/{MAP_NAME}_{uuid}.parquet` (1파일 = 1매치)


In [1]:
import glob, os
import duckdb
import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

BASE_DIR   = r'C:\배그분석'
OUTPUT_DIR = os.path.join(BASE_DIR, 'analysis_output')
os.makedirs(OUTPUT_DIR, exist_ok=True)

DATE_START = '20260211'
DATE_END   = '20260220'
MAP_NAME   = 'Erangel'

# DuckDB 연결 (전역 재사용)
con = duckdb.connect(':memory:')
con.execute('PRAGMA threads=6; PRAGMA memory_limit="8GB";')

print('설정 완료')
print(f'DuckDB version: {duckdb.__version__}')


설정 완료
DuckDB version: 1.4.4


In [2]:
def get_match_parquet_files(map_name, date_start, date_end):
    """날짜 범위 내 매치 parquet 파일 수집
    파일 구조: temp_parquet_files_{server}_{ymd}/{MAP_NAME}_{uuid}.parquet
    """
    files = []
    for date in pd.date_range(date_start, date_end):
        ymd = date.strftime('%Y%m%d')
        for server in ['kakao', 'steam']:
            folder = Path(BASE_DIR) / f'temp_parquet_files_{server}_{ymd}'
            if not folder.exists():
                continue
            files.extend(sorted(folder.glob(f'{map_name}_*.parquet')))
    return files


def extract_match_id(filepath, map_name):
    """파일명에서 matchId(uuid) 추출
    파일명: Erangel_047584e1-2c6e-4ae4-bb66-e263beb868aa.parquet
    -> 047584e1-2c6e-4ae4-bb66-e263beb868aa
    split('_')[1] 은 uuid 안에 _ 가 없을때만 안전
    -> map_name + '_' prefix 제거 방식으로 안전하게 추출
    """
    return Path(filepath).stem[len(map_name) + 1:]  # 'Erangel_' 이후 전체


PARQUETS = get_match_parquet_files(MAP_NAME, DATE_START, DATE_END)
print(f'매치 파일 수: {len(PARQUETS)}')

# matchId 추출 검증
if PARQUETS:
    sample_id = extract_match_id(PARQUETS[0], MAP_NAME)
    print(f'샘플 matchId: {sample_id} (길이: {len(sample_id)})')
    # uuid 형식 검증 (8-4-4-4-12)
    parts = sample_id.split('-')
    if len(parts) == 5:
        print('uuid 형식 정상')
    else:
        print(f'경고: uuid 형식 이상 - {sample_id}')


매치 파일 수: 2224
샘플 matchId: 01a4a247-7e5a-4835-a2f7-a3f7c9fb2aac (길이: 36)
uuid 형식 정상


In [3]:
# STEP 1: 낙하 지점 추출
# LogParachuteLanding 에서 플레이어당 첫 번째만 사용 (재낙하 제외)
print('낙하 지점 추출 중')

drop_list = []
skip_count = 0

for f in tqdm(PARQUETS):
    match_id = extract_match_id(f, MAP_NAME)
    fsql = str(f).replace('\\', '/')

    df = con.execute(f"""
        SELECT
            character_accountId AS accountId,
            character_name      AS playerName,
            character_teamId    AS teamId,
            character_location_x AS x,
            character_location_y AS y,
            _D
        FROM read_parquet('{fsql}')
        WHERE _T = 'LogParachuteLanding'
          AND character_accountId IS NOT NULL
          AND character_location_x IS NOT NULL
    """).fetchdf()

    if df.empty:
        skip_count += 1
        continue

    # 플레이어당 첫 번째 낙하만 사용
    df = df.sort_values('_D').drop_duplicates('accountId', keep='first')
    df['matchId'] = match_id
    drop_list.append(df[['matchId','accountId','playerName','teamId','x','y']])

df_drop = pd.concat(drop_list, ignore_index=True)
print(f'낙하 지점: {len(df_drop):,}명  |  매치: {df_drop["matchId"].nunique():,}개  |  스킵: {skip_count}개')
display(df_drop.head(3))


낙하 지점 추출 중


100%|██████████| 2224/2224 [00:38<00:00, 58.33it/s]


낙하 지점: 148,476명  |  매치: 2,224개  |  스킵: 0개


,matchId,accountId,playerName,teamId,x,y
0,01a4a247-7e5a-4835-a2f7-a3f7c9fb2aac,account.ea72c1bb653648c6877ae1bd79c9af1a,SxxHnxMxx,15.0,179705.703125,630811.875
1,01a4a247-7e5a-4835-a2f7-a3f7c9fb2aac,account.6cd69a93ab7f4fc19730c59f34134261,youudonthavemom,10.0,172562.031250,613398.750
2,01a4a247-7e5a-4835-a2f7-a3f7c9fb2aac,account.e92917d5921845aca318975e77942cb6,MaxDowman,10.0,171455.468750,620891.250


In [4]:
# STEP 2: 비행기 경로 복원 + 낙하 거리 계산
#
# isGame 0.1 구간에서 z값이 최댓값 기준 500 이내인 레코드 = 비행 중
# arg_min/arg_max: DuckDB 0.8+ 지원. 미지원시 ORDER BY + FIRST/LAST 대체
print('비행기 경로 추출 중')

# DuckDB arg_min 지원 여부 확인
try:
    con.execute("SELECT arg_min(1, 1)").fetchone()
    USE_ARGMIN = True
    print(f'arg_min 지원 (DuckDB {duckdb.__version__})')
except Exception:
    USE_ARGMIN = False
    print('arg_min 미지원 -> FIRST/LAST 방식 사용')

air_list = []
skip_count = 0

for f in tqdm(PARQUETS):
    match_id = extract_match_id(f, MAP_NAME)
    fsql = str(f).replace('\\', '/')

    if USE_ARGMIN:
        q = f"""
            WITH base AS (
                SELECT character_location_x AS x,
                       character_location_y AS y,
                       character_location_z AS z,
                       elapsedTime
                FROM read_parquet('{fsql}')
                WHERE _T = 'LogPlayerPosition'
                  AND common_isGame BETWEEN 0.05 AND 0.15
                  AND character_location_x IS NOT NULL
                  AND character_location_z IS NOT NULL
            ),
            flying AS (
                SELECT * FROM base
                WHERE z >= (SELECT MAX(z) FROM base) - 500
            )
            SELECT
                arg_min(x, elapsedTime) AS x_start,
                arg_min(y, elapsedTime) AS y_start,
                arg_max(x, elapsedTime) AS x_end,
                arg_max(y, elapsedTime) AS y_end,
                COUNT(*) AS n_flying
            FROM flying
        """
    else:
        q = f"""
            WITH base AS (
                SELECT character_location_x AS x,
                       character_location_y AS y,
                       character_location_z AS z,
                       elapsedTime
                FROM read_parquet('{fsql}')
                WHERE _T = 'LogPlayerPosition'
                  AND common_isGame BETWEEN 0.05 AND 0.15
                  AND character_location_x IS NOT NULL
                  AND character_location_z IS NOT NULL
            ),
            flying AS (
                SELECT * FROM base
                WHERE z >= (SELECT MAX(z) FROM base) - 500
                ORDER BY elapsedTime
            )
            SELECT
                FIRST(x) AS x_start,
                FIRST(y) AS y_start,
                LAST(x)  AS x_end,
                LAST(y)  AS y_end,
                COUNT(*) AS n_flying
            FROM flying
        """

    df = con.execute(q).fetchdf()

    if df.empty or df['x_start'].isna().all():
        skip_count += 1
        continue

    # 시작점 = 끝점이면 경로 정보 없음 (비행 구간이 너무 짧은 매치)
    if df['x_start'].iloc[0] == df['x_end'].iloc[0] and df['y_start'].iloc[0] == df['y_end'].iloc[0]:
        skip_count += 1
        continue

    df['matchId'] = match_id
    air_list.append(df)

df_air = pd.concat(air_list, ignore_index=True)
print(f'경로 복원: {len(df_air):,}개 매치  |  스킵: {skip_count}개')
display(df_air.head(3))


def point_to_line_dist(px, py, x1, y1, x2, y2):
    """점(px,py)에서 직선 (x1,y1)-(x2,y2)까지 수직 거리"""
    dx, dy = x2 - x1, y2 - y1
    len_sq = dx*dx + dy*dy
    if len_sq == 0:
        return np.sqrt((px-x1)**2 + (py-y1)**2)
    t = ((px-x1)*dx + (py-y1)*dy) / len_sq
    return np.sqrt((px-(x1+t*dx))**2 + (py-(y1+t*dy))**2)


path_dict = df_air.set_index('matchId').to_dict('index')

df_drop['drop_distance_from_path'] = df_drop.apply(
    lambda r: point_to_line_dist(
        r['x'], r['y'],
        path_dict[r['matchId']]['x_start'], path_dict[r['matchId']]['y_start'],
        path_dict[r['matchId']]['x_end'],   path_dict[r['matchId']]['y_end'],
    ) / 100 if r['matchId'] in path_dict else np.nan,  # cm → m
    axis=1
)

valid = df_drop['drop_distance_from_path'].notna().sum()
print(f'낙하 거리 계산: {valid:,}/{len(df_drop):,}')
print(f'평균: {df_drop["drop_distance_from_path"].mean():.1f}m  '
      f'최대: {df_drop["drop_distance_from_path"].max():.1f}m  '
      f'최소: {df_drop["drop_distance_from_path"].min():.1f}m')


비행기 경로 추출 중
arg_min 지원 (DuckDB 1.4.4)


100%|██████████| 2224/2224 [00:08<00:00, 257.11it/s]

경로 복원: 2,221개 매치  |  스킵: 3개


,x_start,y_start,x_end,y_end,n_flying,matchId
0,80804.500,883259.000,602225.8125,172566.18750,254,01a4a247-7e5a-4835-a2f7-a3f7c9fb2aac
1,939323.875,632995.375,114336.8125,483029.71875,231,01a58e2c-d5b5-4741-9c2b-5ca94b6873fb
2,840689.250,789717.250,219559.0000,238775.81250,234,046d6575-7184-4e61-bbc9-6ba9b7e30339


낙하 거리 계산: 148,468/148,476
평균: 775.0m  최대: 3029.4m  최소: 0.0m


In [5]:
# STEP 3: 초기 적 밀도 계산
# isGame 1.0 이상 1.5 미만 구간의 위치 median 기준
# 낙하 지점 반경 500m 내 적 팀 플레이어 수
# BETWEEN 1.0 AND 1.5 는 1.5 포함 -> >= 1.0 AND < 1.5 사용
print('초기 적 밀도 계산 중')

density_rows = []
skip_count   = 0

for f in tqdm(PARQUETS):
    match_id = extract_match_id(f, MAP_NAME)
    fsql = str(f).replace('\\', '/')

    df = con.execute(f"""
        SELECT
            character_accountId AS accountId,
            character_teamId    AS teamId,
            character_location_x AS x,
            character_location_y AS y
        FROM read_parquet('{fsql}')
        WHERE _T = 'LogPlayerPosition'
          AND common_isGame >= 1.0
          AND common_isGame <  1.5
          AND character_accountId IS NOT NULL
          AND character_location_x IS NOT NULL
    """).fetchdf()

    if df.empty:
        skip_count += 1
        continue

    df_med = df.groupby(['accountId','teamId'])[['x','y']].median().reset_index()
    arr   = df_med[['x','y']].to_numpy()
    teams = df_med['teamId'].to_numpy()
    aids  = df_med['accountId'].to_numpy()

    # 낙하 지점 기준으로 계산 (isGame 1.0 초기 위치가 아닌 실제 낙하 좌표)
    match_drop = df_drop[df_drop['matchId'] == match_id][['accountId','teamId','x','y']]

    if match_drop.empty:
        # LogParachuteLanding 이 없는 매치는 초기 위치로 대체
        for i in range(len(df_med)):
            dist = np.sqrt(((arr - arr[i])**2).sum(axis=1))
            enemy_cnt = int(((dist <= 500) & (teams != teams[i])).sum())
            density_rows.append((match_id, aids[i], enemy_cnt))
    else:
        # 낙하 지점 기준 계산
        for _, drow in match_drop.iterrows():
            px, py  = drow['x'], drow['y']
            my_team = drow['teamId']
            aid     = drow['accountId']
            # 초기 위치가 있는 플레이어 중 적 팀만
            mask = teams != my_team
            if mask.sum() == 0:
                enemy_cnt = 0
            else:
                dist = np.sqrt(((arr[mask] - [px, py])**2).sum(axis=1))
                enemy_cnt = int((dist <= 50000).sum())  # 500m = 50000cm
            density_rows.append((match_id, aid, enemy_cnt))

df_density = pd.DataFrame(density_rows, columns=['matchId','accountId','early_enemy_density'])
print(f'완료: {len(df_density):,}명  |  스킵: {skip_count}개  |  평균 적 수(반경 500m): {df_density["early_enemy_density"].mean():.2f}명')
display(df_density.head(3))


초기 적 밀도 계산 중


100%|██████████| 2224/2224 [00:34<00:00, 63.88it/s]


완료: 148,472명  |  스킵: 2개  |  평균 적 수(반경 500m): 5.11명


,matchId,accountId,early_enemy_density
0,01a4a247-7e5a-4835-a2f7-a3f7c9fb2aac,account.ea72c1bb653648c6877ae1bd79c9af1a,11
1,01a4a247-7e5a-4835-a2f7-a3f7c9fb2aac,account.6cd69a93ab7f4fc19730c59f34134261,12
2,01a4a247-7e5a-4835-a2f7-a3f7c9fb2aac,account.e92917d5921845aca318975e77942cb6,12


In [6]:
# STEP 4: 전체 위치 데이터 + 자기장 정보 조인
#
# LogGameStatePeriodic 의 자기장 정보를 LogPlayerPosition 과
# 매치 단위 merge_asof (direction='backward') 로 조인
# isGame >= 1.0 필터 필수 (비행기 구간 제외)
print('위치 + 자기장 조인 중')

pos_all = []
empty_pos = 0
empty_gs  = 0

for f in tqdm(PARQUETS):
    match_id = extract_match_id(f, MAP_NAME)
    fsql = str(f).replace('\\', '/')

    # 1) 플레이어 위치 (isGame >= 1.0 필터 필수)
    df_pos = con.execute(f"""
        SELECT
            character_accountId      AS accountId,
            character_location_x     AS x,
            character_location_y     AS y,
            character_location_z     AS z,
            character_isInBlueZone   AS isInBlueZone,
            character_isInVehicle    AS isInVehicle,
            common_isGame            AS isGame,
            TRY_CAST(elapsedTime AS DOUBLE) AS elapsedTime
        FROM read_parquet('{fsql}')
        WHERE _T = 'LogPlayerPosition'
          AND common_isGame >= 1.0
          AND character_accountId IS NOT NULL
          AND character_location_x IS NOT NULL
          AND TRY_CAST(elapsedTime AS DOUBLE) IS NOT NULL
    """).fetchdf()

    if df_pos.empty:
        empty_pos += 1
        continue

    df_pos['matchId'] = match_id

    # 2) 자기장 정보
    # gameState_elapsedTime 이 없으면 elapsedTime 으로 fallback
    df_gs = con.execute(f"""
        SELECT
            TRY_CAST(
                COALESCE(gameState_elapsedTime, elapsedTime)
            AS DOUBLE) AS gs_time,
            gameState_safetyZonePosition_x AS safe_x,
            gameState_safetyZonePosition_y AS safe_y,
            gameState_safetyZoneRadius     AS safe_radius
        FROM read_parquet('{fsql}')
        WHERE _T = 'LogGameStatePeriodic'
          AND gameState_safetyZonePosition_x IS NOT NULL
          AND TRY_CAST(
                COALESCE(gameState_elapsedTime, elapsedTime)
              AS DOUBLE) IS NOT NULL
    """).fetchdf()

    if df_gs.empty:
        empty_gs += 1
        df_pos[['safe_x','safe_y','safe_radius']] = np.nan
        pos_all.append(df_pos)
        continue

    df_gs  = df_gs.sort_values('gs_time').reset_index(drop=True)
    df_pos = df_pos.sort_values('elapsedTime').reset_index(drop=True)

    # 3) merge_asof: 각 위치 기록에 직전 자기장 정보 조인
    merged = pd.merge_asof(
        df_pos, df_gs,
        left_on='elapsedTime', right_on='gs_time',
        direction='backward'
    ).drop(columns='gs_time')

    # 4) pos 가 gs 보다 이른 구간 (게임 초반) -> 첫 번째 gs 값으로 채움
    first_gs = df_gs.iloc[0]
    merged['safe_x']      = merged['safe_x'].fillna(first_gs['safe_x'])
    merged['safe_y']      = merged['safe_y'].fillna(first_gs['safe_y'])
    merged['safe_radius'] = merged['safe_radius'].fillna(first_gs['safe_radius'])

    pos_all.append(merged)

if not pos_all:
    raise ValueError('pos_all 이 비었습니다. 컬럼명/필터 조건을 확인하세요.')

df_pos_full = pd.concat(pos_all, ignore_index=True)
print(f'조인 완료: {len(df_pos_full):,}행')
print(f'empty pos: {empty_pos}  empty gs: {empty_gs}')
print(f'safe_x null: {df_pos_full["safe_x"].isna().mean():.1%}')

# bool 변환
df_pos_full['isInBlueZone'] = df_pos_full['isInBlueZone'].astype(str).str.lower().isin(['true','1'])
df_pos_full['isInVehicle']  = df_pos_full['isInVehicle'].astype(str).str.lower().isin(['true','1'])

# 안전구역 중심 거리
df_pos_full['dist_to_safe_center'] = np.sqrt(
    (df_pos_full['x'] - df_pos_full['safe_x'])**2 +
    (df_pos_full['y'] - df_pos_full['safe_y'])**2
) / 100  # cm → m
print('처리 완료')


위치 + 자기장 조인 중


100%|██████████| 2224/2224 [00:23<00:00, 93.25it/s]


조인 완료: 11,774,200행
empty pos: 0  empty gs: 0
safe_x null: 0.0%
처리 완료


In [7]:
# STEP 5: 차량 이동 피처
# LogVehicleRide / LogVehicleLeave 위치 기준
# 낙하 지점 대비 최대 이동 거리 계산
# -> 비행기 경로 기준 낙하 거리보다 더 멀리 이동한 사람 = 차량 활용형
print('차량 이동 계산 중')

# 실제 이벤트명 확인 (첫 파일에서)
fsql_sample = str(PARQUETS[0]).replace('\\','/')
df_veh_evt = con.execute(f"""
    SELECT _T, COUNT(*) AS n
    FROM read_parquet('{fsql_sample}')
    WHERE _T LIKE '%Vehicle%'
    GROUP BY 1
""").fetchdf()
print('차량 이벤트명:')
display(df_veh_evt)

VEH_EVENTS = tuple(df_veh_evt['_T'].tolist())

veh_list = []

for f in tqdm(PARQUETS):
    match_id = extract_match_id(f, MAP_NAME)
    fsql = str(f).replace('\\', '/')

    if not VEH_EVENTS:
        continue

    evt_filter = ' OR '.join([f"_T = '{e}'" for e in VEH_EVENTS])

    df = con.execute(f"""
        SELECT
            character_accountId  AS accountId,
            character_location_x AS x,
            character_location_y AS y
        FROM read_parquet('{fsql}')
        WHERE ({evt_filter})
          AND character_accountId IS NOT NULL
          AND character_location_x IS NOT NULL
    """).fetchdf()

    if df.empty:
        continue

    df['matchId'] = match_id

    # 낙하 지점과 merge
    df = df.merge(
        df_drop[['matchId','accountId','x','y']]
               .rename(columns={'x':'drop_x','y':'drop_y'}),
        on=['matchId','accountId'], how='left'
    )

    df['veh_dist'] = np.sqrt(
        (df['x'] - df['drop_x'])**2 +
        (df['y'] - df['drop_y'])**2
    )

    df_max = df.groupby(['matchId','accountId'])['veh_dist'].max().reset_index()
    df_max.rename(columns={'veh_dist':'max_vehicle_distance'}, inplace=True)
    veh_list.append(df_max)

if veh_list:
    df_veh_max = pd.concat(veh_list, ignore_index=True)
    df_veh_max['max_vehicle_distance'] = df_veh_max['max_vehicle_distance'] / 100  # cm → m
    print(f'차량 이동 피처: {len(df_veh_max):,}명  평균: {df_veh_max["max_vehicle_distance"].mean():.1f}m')
else:
    df_veh_max = pd.DataFrame(columns=['matchId','accountId','max_vehicle_distance'])
    print('차량 이벤트 없음')


차량 이동 계산 중
차량 이벤트명:


,_T,n
0,LogVehicleDestroy,4
1,LogVehicleLeave,508
2,LogItemPickupFromVehicleTrunk,23
3,LogItemPutToVehicleTrunk,32
4,LogVehicleRide,509
5,LogVehicleDamage,918


100%|██████████| 2224/2224 [01:23<00:00, 26.66it/s]

차량 이동 피처: 148,490명  평균: 5759.7m


In [8]:
# STEP 6: 플레이어별 피처 집계
print('피처 집계 중')

# rotation_timing_score: 안정 구간 vs 수축 구간 자기장 거리 비율
# 0 = 선점형 (안정 구간에 이미 자기장 안에 있음)
# 1 = 후행형 (수축 구간에 이동)
SHRINK_PHASES = [1.5, 2.5, 3.5, 4.5, 5.5, 6.5]
STABLE_PHASES = [1.0, 2.0, 3.0, 4.0, 5.0]

def calc_rotation_timing(grp):
    pre  = grp[grp['isGame'].isin(STABLE_PHASES)]['dist_to_safe_center'].mean()
    post = grp[grp['isGame'].isin(SHRINK_PHASES)]['dist_to_safe_center'].mean()
    if pd.isna(pre) or pd.isna(post):
        return np.nan
    return pre / (pre + post + 1e-6)


features = []
grouped  = df_pos_full.groupby(['matchId','accountId'])
print(f'총 {grouped.ngroups:,}명 처리 예정')

for (match_id, account_id), grp in tqdm(grouped, total=grouped.ngroups):
    grp = grp.sort_values('elapsedTime')
    safe_r = grp['safe_radius'].replace(0, np.nan)  # 0 으로 나누기 방지

    features.append({
        'matchId':                  match_id,
        'accountId':                account_id,
        'rotation_timing_score':    calc_rotation_timing(grp),
        'vehicle_use_ratio':        grp['isInVehicle'].mean(),
        'bluezone_exposure_ratio':  grp['isInBlueZone'].mean(),
        'safezone_proximity_mean':  grp['dist_to_safe_center'].mean(),
        'safezone_edge_ratio':      (grp['dist_to_safe_center'] / (safe_r + 1e-6)).mean(),
        'altitude_variance':        grp['z'].std(),
        'survival_time':            grp['elapsedTime'].max() - grp['elapsedTime'].min(),
        'total_movement':           np.sqrt(grp['x'].diff()**2 + grp['y'].diff()**2).sum(),
    })

df_features = pd.DataFrame(features)
print(f'피처 테이블: {df_features.shape}')
display(df_features.head(3))


피처 집계 중
총 147,673명 처리 예정


100%|██████████| 147673/147673 [02:42<00:00, 911.54it/s]


피처 테이블: (147673, 10)


,matchId,accountId,rotation_timing_score,vehicle_use_ratio,bluezone_exposure_ratio,safezone_proximity_mean,safezone_edge_ratio,altitude_variance,survival_time,total_movement
0,001491cf-ad60-4ebc-b6a1-c86bc05959fd,account.00ae636f3f29443490716d28017686d2,0.635235,0.172414,0.126437,1177.286963,0.006873,1430.389772,1719.0,6.582388e+05
1,001491cf-ad60-4ebc-b6a1-c86bc05959fd,account.04550355e52549ea99457094928c0316,0.556537,0.147826,0.052174,1486.343883,0.006034,21271.393148,1304.0,9.153429e+05
2,001491cf-ad60-4ebc-b6a1-c86bc05959fd,account.09c7523488a6456abbee4404309fa9b4,0.695087,0.491525,0.000000,782.124023,0.004091,12176.072837,1288.0,1.220069e+06


In [9]:
# STEP 7: 모든 피처 병합
df_all = df_features\
    .merge(
        df_drop[['matchId','accountId','drop_distance_from_path','x','y']]
               .rename(columns={'x':'drop_x','y':'drop_y'}),
        on=['matchId','accountId'], how='left'
    )\
    .merge(
        df_density[['matchId','accountId','early_enemy_density']],
        on=['matchId','accountId'], how='left'
    )\
    .merge(
        df_veh_max,
        on=['matchId','accountId'], how='left'
    )

print(f'통합 피처: {df_all.shape}')
display(df_all.describe())


통합 피처: (147673, 15)


,rotation_timing_score,vehicle_use_ratio,bluezone_exposure_ratio,safezone_proximity_mean,safezone_edge_ratio,altitude_variance,survival_time,total_movement,drop_distance_from_path,drop_x,drop_y,early_enemy_density,max_vehicle_distance
count,120609.000000,147673.000000,147673.000000,147673.000000,147673.000000,147278.000000,147673.000000,1.476730e+05,147653.000000,147661.000000,147661.000000,147657.000000,147661.000000
mean,0.549931,0.139039,0.055366,1486.583650,0.005751,3080.158264,799.060695,3.718595e+05,777.220578,413636.007807,378205.984106,5.075079,5763.756892
std,0.079954,0.143011,0.106280,758.422955,0.222160,5071.617415,493.802542,3.132436e+05,474.647451,158715.959472,137671.878992,5.966501,1689.402908
min,0.070719,0.000000,0.000000,58.681892,0.000101,0.000000,0.000000,0.000000e+00,0.015580,7231.764648,2598.840088,0.000000,224.710265
25%,0.498127,0.000000,0.000000,876.357431,0.003430,352.274918,331.000000,4.926664e+04,374.106604,324285.031250,286448.843750,0.000000,4500.141290
50%,0.545585,0.113924,0.000000,1307.539936,0.004665,1136.899971,832.000000,3.585299e+05,755.233621,414576.281250,348004.812500,4.000000,5657.570220
75%,0.602157,0.237037,0.063636,2018.007035,0.005850,2472.147234,1200.000000,6.007168e+05,1151.037673,549634.687500,474586.625000,8.000000,6997.961189
max,0.973738,1.000000,1.000000,5443.931022,72.325205,54959.436522,1875.000000,2.294886e+06,3029.393315,775228.187500,748743.750000,47.000000,10581.265627


In [10]:
# STEP 8: matches 데이터 조인
# 날짜 필터 적용 (DATE_START ~ DATE_END)
# playerId -> accountId rename 후 조인
print('matches 데이터 로드 중')

mat_files = [
    f for f in sorted(glob.glob(os.path.join(BASE_DIR, 'matches_*.csv')))
    if DATE_START <= os.path.basename(f).replace('.csv','').split('_')[-1] <= DATE_END
]
print(f'matches 파일: {len(mat_files)}개')

df_matches = pd.concat([pd.read_csv(f) for f in mat_files], ignore_index=True)

if 'win_flag' not in df_matches.columns:
    df_matches['win_flag']   = (df_matches['winPlace'] == 1).astype(int)
if 'top3_flag' not in df_matches.columns:
    df_matches['top3_flag'] = (df_matches['winPlace'] <= 10).astype(int)

# playerId -> accountId
df_matches = df_matches.rename(columns={'playerId': 'accountId'})

# 타입 통일
df_all['matchId']       = df_all['matchId'].astype(str)
df_all['accountId']     = df_all['accountId'].astype(str)
df_matches['matchId']   = df_matches['matchId'].astype(str)
df_matches['accountId'] = df_matches['accountId'].astype(str)

df_final = df_all.merge(
    df_matches[['matchId','accountId','kills','damageDealt','winPlace',
                'win_flag','top3_flag','timeSurvived',
                'walkDistance','rideDistance','heals','boosts']],
    on=['matchId','accountId'], how='left'
)

join_rate = df_final['winPlace'].notna().mean()
print(f'최종: {df_final.shape}  조인률: {join_rate:.1%}')

if join_rate < 0.5:
    print('경고: 조인률이 50% 미만입니다. matchId/accountId 형식을 확인하세요.')

display(df_final.head(3))


matches 데이터 로드 중
matches 파일: 20개
최종: (147673, 25)  조인률: 94.1%


,matchId,accountId,rotation_timing_score,vehicle_use_ratio,bluezone_exposure_ratio,safezone_proximity_mean,safezone_edge_ratio,altitude_variance,survival_time,total_movement,...,kills,damageDealt,winPlace,win_flag,top3_flag,timeSurvived,walkDistance,rideDistance,heals,boosts
0,001491cf-ad60-4ebc-b6a1-c86bc05959fd,account.00ae636f3f29443490716d28017686d2,0.635235,0.172414,0.126437,1177.286963,0.006873,1430.389772,1719.0,6.582388e+05,...,3.0,249.50539,2.0,0.0,1.0,1766.0,4123.179,4482.5470,4.0,6.0
1,001491cf-ad60-4ebc-b6a1-c86bc05959fd,account.04550355e52549ea99457094928c0316,0.556537,0.147826,0.052174,1486.343883,0.006034,21271.393148,1304.0,9.153429e+05,...,0.0,0.00000,4.0,0.0,1.0,1176.0,2180.932,5912.9204,1.0,5.0
2,001491cf-ad60-4ebc-b6a1-c86bc05959fd,account.09c7523488a6456abbee4404309fa9b4,0.695087,0.491525,0.000000,782.124023,0.004091,12176.072837,1288.0,1.220069e+06,...,1.0,329.19660,5.0,0.0,1.0,1198.0,1521.188,10001.2380,2.0,6.0


In [11]:
# STEP 9: 저장 및 최종 검증
out = os.path.join(OUTPUT_DIR, 'erangel_features.parquet')
df_final.to_parquet(out, index=False)

print(f'저장 완료: {out}')
print(f'크기: {df_final.shape[0]:,}행 x {df_final.shape[1]}열')
print()

# 결측치 현황
missing = df_final.isnull().mean().sort_values(ascending=False)
missing_df = missing[missing > 0].rename('결측비율').map('{:.1%}'.format).to_frame()
print('결측치 현황:')
display(missing_df)

# 핵심 피처 기초 통계
key_features = [
    'drop_distance_from_path', 'early_enemy_density',
    'rotation_timing_score', 'vehicle_use_ratio',
    'bluezone_exposure_ratio', 'safezone_proximity_mean',
    'safezone_edge_ratio', 'altitude_variance',
    'max_vehicle_distance',
]
available = [c for c in key_features if c in df_final.columns]
print('\n핵심 피처 기초 통계:')
display(df_final[available].describe().T)

print('\n02_clustering.ipynb 로')


저장 완료: C:\배그분석\analysis_output\erangel_features.parquet
크기: 147,673행 x 25열

결측치 현황:


,결측비율
rotation_timing_score,18.3%
winPlace,5.9%
win_flag,5.9%
top3_flag,5.9%
timeSurvived,5.9%
walkDistance,5.9%
rideDistance,5.9%
damageDealt,5.9%
kills,5.9%
heals,5.9%



핵심 피처 기초 통계:


,count,mean,std,min,25%,50%,75%,max
drop_distance_from_path,147653.0,777.220578,474.647451,0.015580,374.106604,755.233621,1151.037673,3029.393315
early_enemy_density,147657.0,5.075079,5.966501,0.000000,0.000000,4.000000,8.000000,47.000000
rotation_timing_score,120609.0,0.549931,0.079954,0.070719,0.498127,0.545585,0.602157,0.973738
vehicle_use_ratio,147673.0,0.139039,0.143011,0.000000,0.000000,0.113924,0.237037,1.000000
bluezone_exposure_ratio,147673.0,0.055366,0.106280,0.000000,0.000000,0.000000,0.063636,1.000000
safezone_proximity_mean,147673.0,1486.583650,758.422955,58.681892,876.357431,1307.539936,2018.007035,5443.931022
safezone_edge_ratio,147673.0,0.005751,0.222160,0.000101,0.003430,0.004665,0.005850,72.325205
altitude_variance,147278.0,3080.158264,5071.617415,0.000000,352.274918,1136.899971,2472.147234,54959.436522
max_vehicle_distance,147661.0,5763.756892,1689.402908,224.710265,4500.141290,5657.570220,6997.961189,10581.265627



02_clustering.ipynb 로
